In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score
import numpy as np
import cv2
from torchinfo import summary  # Model summary
from torchvision.models import VGG16_Weights
from tqdm import tqdm  # Progress bar

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Set Paths to the dataset
dataset_path_train = '/data/leuven/361/vsc36144/maibi_cv/Vision+/train'
dataset_path_val = '/data/leuven/361/vsc36144/maibi_cv/Vision+/val'
dataset_path_test = '/data/leuven/361/vsc36144/maibi_cv/Vision+/test'

# Set Parameters for Training
batch_size = 32
epochs = 10
input_shape = (224, 224)

# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets
train_dataset = datasets.ImageFolder(dataset_path_train, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(dataset_path_val, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(dataset_path_test, transform=data_transforms['test'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained VGG16 model
base_model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[0].in_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, len(train_dataset.classes)),
    nn.Softmax(dim=1)
)
base_model.to(device)

# Freeze the layers of the pre-trained model
for param in base_model.features.parameters():
    param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.classifier.parameters())


In [ ]:
# Training and validation loop with progress bar
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Progress bar for training
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                pbar.set_postfix({"Loss": running_loss / (pbar.n + 1), "Accuracy": correct / total})
                pbar.update()

        train_accuracy = correct / total
        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader)}, Train Accuracy: {train_accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

    return model

# Evaluation function
def evaluate_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss/len(loader), correct/total

In [ ]:
# Train the model
trained_model = train_model(base_model, train_loader, val_loader, criterion, optimizer, epochs=epochs)

# Display the model summary
summary(trained_model, input_size=(batch_size, 3, input_shape[0], input_shape[1]))

# Save the trained model
torch.save(trained_model.state_dict(), '/data/leuven/361/vsc36144/maibi_cv/Saved_Model/openface_model.pth')

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Calculate F1 score on the test dataset
def calculate_f1(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return f1_score(y_true, y_pred, average='weighted')

f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score: {f1score}")

In [ ]:
# Evaluate the model on the Currupted test dataset
dataset_path_test_currupt_1 = '/data/leuven/361/vsc36144/maibi_cv/Vision+/corrupted_test/test_s_1_corr=gaussian_blur'
dataset_path_test_currupt_2 = '/data/leuven/361/vsc36144/maibi_cv/Vision+/corrupted_test/test_s_2_corr=gaussian_blur'
dataset_path_test_currupt_3 = '/data/leuven/361/vsc36144/maibi_cv/Vision+/corrupted_test/test_s_3_corr=gaussian_blur'
dataset_path_test_currupt_4 = '/data/leuven/361/vsc36144/maibi_cv/Vision+/corrupted_test/test_s_4_corr=gaussian_blur'
dataset_path_test_currupt_5 = '/data/leuven/361/vsc36144/maibi_cv/Vision+/corrupted_test/test_s_5_corr=gaussian_blur'

# Load datasets
test_dataset_1 = datasets.ImageFolder(dataset_path_test_currupt_1, transform=data_transforms['test'])
test_dataset_2 = datasets.ImageFolder(dataset_path_test_currupt_2, transform=data_transforms['test'])
test_dataset_3 = datasets.ImageFolder(dataset_path_test_currupt_3, transform=data_transforms['test'])
test_dataset_4 = datasets.ImageFolder(dataset_path_test_currupt_4, transform=data_transforms['test'])
test_dataset_5 = datasets.ImageFolder(dataset_path_test_currupt_5, transform=data_transforms['test'])

# Calculate F1 score on the test dataset
def calculate_f1(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return f1_score(y_true, y_pred, average='weighted')

# Create data loaders for TestDataset 1
test_loader = DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test_1 Loss: {test_loss}")
print(f"Test_1 Accuracy: {test_accuracy}")
f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score_1: {f1score}")

# Create data loaders for TestDataset 2
test_loader = DataLoader(test_dataset_2, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test_2 Loss: {test_loss}")
print(f"Test_2 Accuracy: {test_accuracy}")
f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score_2: {f1score}")

# Create data loaders for TestDataset 3
test_loader = DataLoader(test_dataset_3, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test_3 Loss: {test_loss}")
print(f"Test_3 Accuracy: {test_accuracy}")
f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score_3: {f1score}")

# Create data loaders for TestDataset 4
test_loader = DataLoader(test_dataset_4, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test_4 Loss: {test_loss}")
print(f"Test_4 Accuracy: {test_accuracy}")
f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score_4: {f1score}")

# Create data loaders for TestDataset 5
test_loader = DataLoader(test_dataset_5, batch_size=batch_size, shuffle=False)
test_loss, test_accuracy = evaluate_model(trained_model, test_loader, criterion)
print(f"Test_5 Loss: {test_loss}")
print(f"Test_5 Accuracy: {test_accuracy}")
f1score = calculate_f1(trained_model, test_loader)
print(f"F1 Score_5: {f1score}")

In [ ]:
# Load the Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the trained OpenFace model
trained_model.load_state_dict(torch.load('/data/leuven/361/vsc36144/maibi_cv/Saved_Model/openface_model.pth'))
trained_model.to(device)
trained_model.eval()

# Define a function to detect and recognize facial expressions
def detect_expression(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        face = cv2.resize(face, input_shape)
        face = transforms.ToTensor()(face).unsqueeze(0)
        face = face.to(device)

        # Predict the emotion class
        with torch.no_grad():
            outputs = trained_model(face)
            _, predicted = torch.max(outputs, 1)
            emotion_class = train_dataset.classes[predicted.item()]

        # Draw bounding box and label on the image
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, emotion_class, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image


In [ ]:
# Load an input image
input_image = cv2.imread('Ahmad.JPG')

# Detect and recognize facial expressions
output_image = detect_expression(input_image)

# Display the output image
cv2.imshow('Facial Expression Recognition', output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()